In [1]:
! pip install datasets transformers

In [2]:
import torch

In [3]:
import transformers
from datasets import load_dataset
from datasets import DatasetDict
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
import math

In [4]:
print(transformers.__version__)

4.28.1


In [5]:
datasets = load_dataset('flax-sentence-embeddings/stackexchange_math_jsonl')

No config specified, defaulting to: stackexchange_math_jsonl/title_answer
Found cached dataset stackexchange_math_jsonl (/home/sslashinin/.cache/huggingface/datasets/flax-sentence-embeddings___stackexchange_math_jsonl/title_answer/1.1.0/6d8702724ae9d14595d55b555648bb7a4909ce0441581f0c5a81e742633441a6)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'upvoted_answer'],
        num_rows: 1100953
    })
})

In [7]:
used_dataset, unused_dataset= datasets['train'].train_test_split(test_size=0.8).values()

In [8]:
used_dataset

Dataset({
    features: ['title', 'upvoted_answer'],
    num_rows: 220190
})

In [9]:
#train_dataset, testval_dataset= datasets['train'].train_test_split(test_size=0.2).values()
train_dataset, testval_dataset = used_dataset.train_test_split(test_size=0.5).values()

In [10]:
test_dataset, val_dataset = testval_dataset.train_test_split(test_size=0.5).values()

In [11]:
datasets_full = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': val_dataset})

In [12]:
datasets_full['train']

Dataset({
    features: ['title', 'upvoted_answer'],
    num_rows: 110095
})

In [13]:
datasets_full

DatasetDict({
    train: Dataset({
        features: ['title', 'upvoted_answer'],
        num_rows: 110095
    })
    test: Dataset({
        features: ['title', 'upvoted_answer'],
        num_rows: 55047
    })
    validation: Dataset({
        features: ['title', 'upvoted_answer'],
        num_rows: 55048
    })
})

In [14]:
datasets_full["train"][10]

{'title': 'Find the maximum elements that the set can contain.',
 'upvoted_answer': 'Fill in the work yourself:\n\nNaively, how many distinct sums could there be? \n\nHence, conclude that there are at most 6 elements.   \n\nIf we had sums of 17 and 3, then we must have the elements $\\{ 1, 2, 8, 9 \\}$.\n\nHence, conclude that there are at most 5 elements.\n\nFind a subset of 5 elements that satisfies the conditions.'}

In [15]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [16]:
show_random_elements(datasets_full["train"])

,title,upvoted_answer
0,If $f_n$ $\longrightarrow$ $f$ in measure and $g_n$ $\longrightarrow$ $g$ in measure then show that $f_n +g_n$ $\longrightarrow$ $f+g$ in measure,Hints for (2):\n\n\nLet $(h_n)_{n \in \mathbb{N}}$ be a sequence of measurable functions. Prove that $h_n \to 0$ in measure implies $h_n^2 \to 0$ in measure.\nUse the first part of your problem and Step 1 to show that $(f_n+g_n)^2 \to 0$ in measure and $(f_n-g_n)^2 \to 0$ in measure.\nUse $$f_n \cdot g_n = \frac{1}{4} \big( (f_n+g_n)^2- (f_n-g_n)^2 \big)$$ and Step 1 to conclude that $f_n \cdot g_n \to 0$ in measure.
1,"If $P(A) \neq 0$ and $P(B) \neq 0$, then $P(B|A) \geq P(B)$ is equivalent to $P(A|B) \geq P(A)$","Both statements are saying that $P(A \cap B) \ge P(A)\cdot P(B)$. Note that $P(A)\cdot P(B)$ corresponds to $P(A \cap B)$ if $A$ and $B$ were independent. Thus $P(A \cap B) \ge P(A)\cdot P(B)$ means that there is some positive correlation (in a figurative sense) between these two events.\n\nOn the other hand, the phrase that always pops in my mind when I am talking about conditional probability is you are changing your universe. $P(X|Y)$ means that you are looking at $P(X)$ in a different universe, i.e. $Y$. Since there is a positive correlation between these two, it makes sense that $P(A|B) \ge P(A)$. This is because we are changing our universe to $B$, and since we know that $B$ has a positive correlation with $A$, it means that in this new universe, we are more likely to see $A$. This is symmetric in $A$ and $B$ of course.\n\nI don't know if this is intuition enough."
2,Solving integral of rectangular function,"Rewriting the given functions in terms of unit step-functions:\n$$\mathrm{rect}_T(t) = u(t+T/2)-u(t-T/2)$$and\n$$\mathrm{rect}_{2T}(t) = u(t+T) - u(t-T),$$where\n$$u(t) = \begin{cases}1, &amp;&amp; t\geq 0\\\n0, &amp;&amp; t&lt;0\end{cases}$$\nNow $$s(t) = \mathrm{rect}_T(t)\star\mathrm{rect}_{2T}(t)$$\n$$=[u(t+T/2) - u(t-T/2)]\star[u(t+T)-u(t-T)]$$\n$$=u(t+T/2)\star u(t+T) - u(t+T/2)\star u(t-T) - u(t-T/2)\star u(t+T) + u(t-T/2)\star u(t-T)$$\nby using the distributive law for convolutions.\n\nUsing the property $f(t)\star g(t) = F(t)\implies f(t-t_1)\star g(t-t_2)=F(t-t_1-t_2)$ and the fact that $u(t)\star u(t) = tu(t)$, we get\n$$s(t) = (t+3T/2)u(t+3T/2)-(t-T/2)u(t-T/2)-(t+T/2)u(t+T/2)+(t-3T/2)u(t-3T/2)$$\nMathematica gives the following result for $s(t)$ for $T=4$:"
3,Why can't one cancel partial differentials?,"Writing the formula like this is wrong for $z\equiv z(x,y), x\equiv x(s,t), y\equiv y(s,t)$\nOne should write it using partial derivatives.\n\begin{align*}\ndz &amp;= \frac{\partial z}{\partial x}dx+ \frac{\partial z}{\partial y}dy\\\ndz &amp;= \frac{\partial z}{\partial x}\left(\frac{\partial x}{\partial s}ds+\frac{\partial x}{\partial t}dt \right)+ \frac{\partial z}{\partial y}\left(\frac{\partial y}{\partial s}ds+\frac{\partial y}{\partial t}dt\right)\tag{1}\\\n\frac{\partial z}{\partial s} &amp;= \left(\frac{\partial z}{\partial x}\frac{\partial x}{\partial s}+ \frac{\partial z}{\partial y}\frac{\partial y}{\partial s}\right)ds+\left(\frac{\partial z}{\partial x}\frac{\partial x}{\partial t}+ \frac{\partial z}{\partial y}\frac{\partial y}{\partial t}\right)dt\\\n\frac{\partial z}{\partial s} &amp;= \frac{\partial z}{\partial x}\frac{\partial x}{\partial s}+ \frac{\partial z}{\partial y}\frac{\partial y}{\partial s}\tag{2}\\\n\end{align*}\nIf it were $z\equiv z(x,y), x\equiv x(s), y\equiv y(s)$, then it would have become\n\begin{align*}\ndz &amp;= \frac{\partial z}{\partial x}\frac{dx}{ds}ds+ \frac{\partial z}{\partial y}\frac{dy}{ds}ds\\\n\frac{dz}{ds} &amp;= \frac{\partial z}{\partial x}\frac{dx}{ds}+ \frac{\partial z}{\partial y}\frac{dy}{ds}\\\n\end{align*}\nIf it were $z\equiv z(x), x\equiv x(s,t)$, then it would have become\n\begin{align*}\ndz &amp;= \frac{dz}{dx}\left(\frac{\partial x}{\partial s}ds+\frac{\partial x}{\partial t}dt \right)\\\n\frac{\partial z}{\partial s} &amp;= \frac{dz}{d\nx}\frac{\partial x}{\parti

In [17]:
model_checkpoint = "facebook/galactica-125M"

In [18]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [19]:
# def tokenize_function(examples):
#     return tokenizer(examples["upvoted_answer"])

def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["upvoted_answer"]], truncation=True)

In [20]:
tokenized_datasets = datasets_full.map(preprocess_function, batched=True, num_proc=4, remove_columns=datasets_full["train"].column_names)

Map (num_proc=4):   0%|          | 0/110095 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map (num_proc=4):   0%|          | 0/55047 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map (num_proc=4):   0%|          | 0/55048 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [21]:
# block_size = tokenizer.model_max_length

block_size = 128

In [67]:
# tokenized_datasets.save_to_disk('.')

In [68]:
# from datasets import load_from_disk
# train_dataset = load_from_disk('/home/sslashinin/kovakimyan/train')
# test_dataset = load_from_disk('/home/sslashinin/kovakimyan/test')
# validation_dataset = load_from_disk('/home/sslashinin/kovakimyan/validation')

In [69]:
# train_dataset

In [70]:
# test_dataset

In [71]:
# validation_dataset

In [22]:

block_size = 128
def group_texts(examples):
    print(examples)
    # Concatenate all texts.
    #for k in examples.keys():
    #  print(k)
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [23]:
#!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [23]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/110095 [00:00<?, ? examples/s]

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [24]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' t ;   3 $,   t h e r e   i s   a   p r i m e   b e t w e e n   $ n $   a n d   $ 2 n $.   I n   p a r t i c u l a r,   t h i s   m e a n s   t h a t   g i v e n   a   p r i m e   $ p _ 0 $,   t h e r e   i s   a n o t h e r   p'

In [75]:
# torch.cuda.empty_cache()

In [76]:
!pwd

/home/sslashinin/kovakimyan


In [77]:
from pathlib import Path
from datasets import DatasetDict, Dataset

# Set the paths to the train, test, and val folders
train_path = Path("/home/sslashinin/kovakimyan/train")
test_path = Path("/home/sslashinin/kovakimyan/test")
val_path = Path("/home/sslashinin/kovakimyan/val")

# Define a function to read the text files and return a dictionary
def read_files(path):
    files = sorted(path.glob("*.txt"))
    data = {"text": [], "label": []}
    for file in files:
        with open(file, "r") as f:
            text = f.read()
        label = file.stem.split("_")[0]
        data["text"].append(text)
        data["label"].append(label)
    return data

# Read the data from the train, test, and val folders
train_data = read_files(train_path)
test_data = read_files(test_path)
val_data = read_files(val_path)

# Convert the data to datasets
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)
val_dataset = Dataset.from_dict(val_data)

# Combine the datasets into a DatasetDict
data_dict = DatasetDict({"train": train_dataset, "test": test_dataset, "validation": val_dataset})

In [26]:
!pwd

/home/sslashinin/kovakimyan


In [25]:
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
#model = AutoModelForCausalLM.from_pretrained("/home/sslashinin/kovakimyan/my_model")

In [26]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    num_train_epochs=7, 
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=15,  # batch size per device during training
    per_device_eval_batch_size=20,
)

In [81]:
# import evaluate
# import sklearn

In [82]:
# metric = evaluate.load("accuracy")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

In [83]:
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

In [84]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    #compute_metrics=compute_metrics,
)
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=validation_dataset,
# )


In [ ]:
trainer.train()

/home/sslashinin/.conda/envs/kovakimyan/lib/python3.11/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


/home/sslashinin/.conda/envs/kovakimyan/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/sslashinin/.conda/envs/kovakimyan/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/sslashinin/.conda/envs/kovakimyan/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/sslashinin/.conda/envs/kovakimyan/lib/python3.11/site-packages/torch/nn/parallel/_fun

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.save_model("./my_model3")

ImportError: cannot import name 'prepare_model_for_fp16_training' from 'peft' (/home/sslashinin/.conda/envs/kovakimyan/lib/python3.11/site-packages/peft/__init__.py)